# Applied Data Science Capstone - Week3 Assignment
### Segmentation and Clustering Neighborhoods in Toronto
In this assignment, we will explore, segment, and cluster the neighborhoods in the city of Toronto.

In [1]:
#importing Packages
from bs4 import BeautifulSoup
import requests
import lxml.html as lh
import pandas as pd

In [55]:
#Linking URL to string url
url = 'http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# 2. Scraping from Wikipedia from requests
page = requests.get(url).text

#Store the contents of the website under doc
doc = BeautifulSoup(page,'lxml')

In [56]:
#Looking table tag and its content from html body
table_post = doc.find('table')
fields = table_post.find_all('td')

postcode = []
borough = []
neighbourhood = []

#Appending values to each column
for i in range(0, len(fields), 3):
    postcode.append(fields[i].text.strip())
    borough.append(fields[i+1].text.strip())
    neighbourhood.append(fields[i+2].text.strip())

#Creating a dataframe
df = pd.DataFrame(data=[postcode, borough, neighbourhood]).transpose()
df.columns = ['Postcode', 'Borough', 'Neighbourhood']
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [57]:
#Removing Rows where Borough values are not assigned
import numpy as np
df['Borough'].replace('Not assigned', np.nan,inplace=True)
df.dropna(subset=['Borough'], inplace=True)

In [58]:
#Combing the rows having same post code and Borough
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df.columns = ['Postcode', 'Borough', 'Neighbourhood']
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [59]:
df.shape

(103, 3)

In [62]:
#Removing Duplicates if any
df_final = df.drop_duplicates(subset=['Postcode', 'Neighbourhood','Borough'], keep=False)
df_final.shape

(103, 3)

In [63]:
df_final.to_csv("Toronto_Data.csv")